In [ ]:
'''File to construct a pretty graph for windshelter'''

from distutils.ccompiler import new_compiler
import numpy as np
import math

# from yaml import DirectiveToken
from elevation import getElevationMatrix, rasterToImage, getRasterRGB
import mercantile
import basic_weather_calls
import wind_shelter
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly.graph_objects as go

import pandas as pd
import numpy as np
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

def wind_shelter_all_data(lat,lng,zoom):
    #creating elevation matrix
    MAPBOX_TOKEN = 'pk.eyJ1IjoiY3Jpc3BpYW5tIiwiYSI6ImNsMG1oazJhejE0YzAzZHVvd2Z1Zjlhb2YifQ.cv0zlPYY6WnoKM9YLD1lMQ'

    tile_coords = mercantile.tile(lng, lat, zoom)
    elevation_mat = getElevationMatrix(MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y)
    #finding wind direction at coords
    direction = np.pi/180*basic_weather_calls.wind_direction(lat,lng)
    
    direction = -direction
    
    direction = -1.71042
    
    #print(direction)
    
    #initial values
    
    tolerance = 30*np.pi/180 #from first paper
    
    #calculation of cellsize
    latitude_radians = lat * math.pi / 180
    
    cellsize = abs(156543.03 * np.cos(latitude_radians) / (2 ** zoom))
    
    
    #using a max dist of 100m (from paper), calculating radius by zoom
    radius = math.ceil(1000/cellsize)
    
    #creating mask for windward direction
    mask = wind_shelter.wind_shelter_prep(radius,direction,tolerance)
    
    #calculating wind shelter
    shelter, shelter_array = wind_shelter.shelter_index(elevation_mat,mask,radius,cellsize,array_return=True)
    
    return shelter,mask,shelter_array,direction


    
# 51.14490081772667, -2.698696870162613

# , , , -2.724194769448233

#using some coords from snowdon
lat = 51.141818132422706
lon = -2.724194769448233
zoom = 12
    
shelter_index,mask,shelter_array,direction = wind_shelter_all_data(lat,lon,zoom)


# print(shelter_index)

# print(direction)


#retrieving elevation matrix, windshelter function does something funny to it
MAPBOX_TOKEN = 'pk.eyJ1IjoiY3Jpc3BpYW5tIiwiYSI6ImNsMG1oazJhejE0YzAzZHVvd2Z1Zjlhb2YifQ.cv0zlPYY6WnoKM9YLD1lMQ'

tile_coords = mercantile.tile(lon, lat, zoom)
elevation_mat = getElevationMatrix(MAPBOX_TOKEN, tile_coords.z, tile_coords.x, tile_coords.y)

# Set up grid
nx = elevation_mat.shape[0]
ny = elevation_mat.shape[1]

mask_overlay = np.ones([nx,ny])

#finding the centre value of the mask
ctr_mask,coord_mask = wind_shelter.centervalue(mask_overlay)

direction = direction
#the mask is a (radius +1 by radius +1) array - we want to iterate from the (centre-radius,centre-radius)
#and reassign all the ones with zeros from the mask 


radius = (mask.shape[0]-1)/2

start_point_i = coord_mask-radius
start_point_j = coord_mask - radius

full_shelter_array = np.zeros([nx,ny])

for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        i_index = int(start_point_i+i)
        j_index = int(start_point_j+j)
        mask_overlay[i_index,j_index]=mask[i,j]
        full_shelter_array[i_index,j_index] = shelter_array[i,j]
        

location = np.where(full_shelter_array==shelter_index)

defining_point = (location[0][0],location[1][0])

data_mask = np.zeros([nx,ny])
data_mask_removed = np.zeros([nx,ny])

for i in range(mask_overlay.shape[0]):
    for j in range(mask_overlay.shape[1]):
        if mask_overlay[i,j] == 1:
            data_mask[i,j]=np.nan
            data_mask_removed[i,j] = elevation_mat[i,j]
        else:
            data_mask[i,j]=elevation_mat[i,j]+0.01
            data_mask_removed[i,j] = elevation_mat[i,j]




radius = 2*int(radius)

# data = elevation_mat
data = elevation_mat[coord_mask-radius:coord_mask+radius,coord_mask-radius:coord_mask+radius]

mask_overlay = mask_overlay[coord_mask-radius:coord_mask+radius,coord_mask-radius:coord_mask+radius]

full_shelter_array_crop = full_shelter_array[coord_mask-radius:coord_mask+radius,coord_mask-radius:coord_mask+radius]

data_mask = data_mask[coord_mask-radius:coord_mask+radius,coord_mask-radius:coord_mask+radius]

data_mask_removed = data_mask_removed[coord_mask-radius:coord_mask+radius,coord_mask-radius:coord_mask+radius]

location = np.where(full_shelter_array_crop==shelter_index)

defining_point_crop = (location[0][0],location[1][0])

colour_tuple = ('y','b','r')
colors = np.empty((mask_overlay.shape[0],mask_overlay.shape[1]), dtype=str)

for i in range(mask_overlay.shape[0]):
    for j in range(mask_overlay.shape[1]):
        if mask_overlay[i,j]==0:
            colors[i,j]=colour_tuple[0]
        elif mask_overlay[i,j]==2:
            colors[i,j]=colour_tuple[2]
        elif mask_overlay[i,j]==1:
            colors[i,j]= colour_tuple[1]

#changing centre point so it can be a different color


x = range(data_mask_removed.shape[0])
y= range(data_mask_removed.shape[1])


X, Y = np.meshgrid(x, y)  # `plot_surface` expects `x` and `y` data to be 2D







In [ ]:
centre_mask,centre_coord_mask = wind_shelter.centervalue(mask_overlay)

scatter_df = pd.DataFrame
scatter_df = {'point':['Point of interest','Shelter defining point'],
            'X':[centre_coord_mask,defining_point_crop[1]],'Y':[centre_coord_mask,defining_point_crop[0]],
              'Z':[data[centre_coord_mask,centre_coord_mask],data[defining_point_crop[0],defining_point_crop[1]]],
            'color':['#0094FF','#FF0000']}

In [ ]:
import seaborn as sns
palett = sns.color_palette("husl", n_colors=3).as_hex()
list(palett)
palett[2] = '#fffB00'
palett[0] = '#000000'

In [ ]:
centre_mask,centre_coord_mask = wind_shelter.centervalue(mask_overlay)






In [ ]:

u = np.sin(direction)
v = np.cos(direction)

wind_direction = np.r_[u, v]


grad = np.gradient(data_mask_removed)


w_array = -(grad[0] * u + grad[1]*v)

u_array = u*np.ones((w_array.shape[0],w_array.shape[1]))
v_array = v*np.ones((w_array.shape[0],w_array.shape[1]))


In [ ]:
data_mask_removed.shape

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
# pio.renderers.default = "notebook"
import pandas as pd

# Read data from a csv
#x=1.9 and 3
# z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
config = {
  'toImageButtonOptions': { 'height': None, 'width': None, 'format':'svg'}
}


fig = go.Figure(data=[
   
    go.Contour(
      z=np.flipud(data_mask_removed),
      line_smoothing=0,
      contours=dict(
            # coloring ='heatmap',
            showlabels = False, # show labels on contours
            labelfont = dict( # label font properties
                size = 10,
                color = 'white',
        )),
      colorbar=dict(
        title="Elevation",
        thickness=25,
        yanchor="bottom",
        y = 0,
        # x=1.2,
        thicknessmode='pixels',
        len=0.8,
        lenmode='fraction',
        outlinewidth=0
      )
    ),

    go.Contour(
      z=np.flipud(data_mask),
      line_smoothing=1,
      showscale=False,
      colorscale=[[0, 'lightgrey'],[1, 'lightgrey']],
      # colorbar=dict(
      #   title="Sheltered",
        
      #   # y=0.5,
      #   thickness=40,
      #   thicknessmode='pixels',
      #   len=0.6,
      #   lenmode='fraction',
      #   outlinewidth=0
      # )
    ),
      
    go.Scatter(name='Specified Coordinates',
      x=np.array(scatter_df['X'][0]),
      y=np.array(168-scatter_df['Y'][0]),
      # z=np.array(scatter_df['Z'][0]),
      mode = 'markers',marker=dict( color=scatter_df['color'][0],size = 12)),
    
    go.Scatter(name='Shelter Defining Point',
      x=np.array(scatter_df['X'][1]),
      y=np.array(168-scatter_df['Y'][1]),
      # z=np.array(scatter_df['Z'][1]),
      mode = 'markers',marker=dict( color=scatter_df['color'][1],size = 12)),

        go.Scatter(name='Searched Area',
    x=np.array(58),
    y=np.array(85),
    # z=np.array(scatter_df['Z'][0]),
    mode = 'markers',
    marker_symbol= 'square',
    marker=dict(color='lightgrey',size = 12)),
  
])


fig.update_layout(title='', autosize=True,
                  height=320,width=800,
                  margin=dict(l=10, r=10, b=10, t=10))
# fig.update_layout(legend_orientation="")
# fig.update_traces(marker={'size': 12})
# fig.update_traces(marker=dict(size=6,
#                          line=dict(width=5,
#                          color='Black')),
#               selector=dict(mode='markers'))
fig.show()

In [ ]:
data_mask

In [ ]:
import os

if not os.path.exists("poster"):
    os.mkdir("poster")

In [ ]:
fig.write_image("poster/crisp_wind_shelter.pdf")

In [ ]:
x = range(data_mask_removed.shape[0])
y= range(data_mask_removed.shape[1])

x_array,y_array = np.meshgrid(x,y)


print(x_array)

print(w_array)

In [ ]:
df = pd.DataFrame

df= {'x':x_array.flatten(),'y':y_array.flatten(),'z':data_mask_removed.flatten(),'u':u.flatten(),'v':v.flatten(),'w':w_array.flatten()}

In [ ]:

import plotly.graph_objs as go

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/streamtube-wind.csv').drop(['Unnamed: 0'],axis=1)

data = [go.Streamtube(
    x = df['x'],
    y = df['y'],
    z = df['z'],
    u = df['u'],
    v = df['v'],
    w = df['w'],starts = dict(
        x = df['x'][1:14],
        y = df['y'][1:14],
        z = df['z'][1:14]),
    sizeref = 0.3,
    colorscale = 'Portland',
    showscale = False,
    maxdisplayed = 1000)]

fig = go.Figure(data=data)

fig.show()



In [ ]:
import plotly.figure_factory as ff
x = range(data_mask_removed.shape[0])
y= range(data_mask_removed.shape[1])


surface_x, surface_y = np.meshgrid(x, y)  # `plot_surface` expects `x` and `y` data to be 2D

u = np.cos(direction)*np.ones((surface_x.shape[0],surface_x.shape[1]))
v = w_array

fig = ff.create_streamline(x, y, u, v)
fig.show()

In [ ]:
print(elevation_mat[1][1:4])

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/streamtube-wind.csv').drop(['Unnamed: 0'],axis=1)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(
    data=[go.Bar(y=[2, 1, 3])],
    layout_title_text="A Figure Displayed with fig.show()"
)
fig